In [1]:
from tensorflow.keras.models import load_model

In [2]:
from tensorflow.keras.preprocessing import image

In [3]:
import tensorflow as tf

In [4]:
import numpy as np
from PIL import Image

In [35]:
model = tf.keras.models.load_model("C:/Users/Alaa/Data_Science/Projects/graduation/Training/pose_estimation/my_model.h5")

In [6]:
model_classes = ['fight', 'nofight']

# prepare Image for prediction using pose estimation

In [10]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2 as cv
from matplotlib import pyplot as plt
import numpy as np

In [37]:
model_pose = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
movenet = model_pose.signatures['serving_default']

In [16]:
def prepare(img):
    # Resize image
    img = img.copy()
    img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 192,256)
    input_img = tf.cast(img, dtype=tf.int32)

    # Detection section
    results = movenet(input_img)
    keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))


    #make a blank img to render pose on it
    blank = np.zeros((360, 480, 3),dtype=np.uint8)

    # Render keypoints 
    loop_through_people(blank, keypoints_with_scores, EDGES, 0.1)

    return blank


In [12]:
# Function to loop through each person detected and render
def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)

In [13]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv.circle(frame, (int(kx), int(ky)), 3, (0,255,0), -1)

In [14]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [15]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 4)

# predictions

In [17]:
import cv2 as cv

In [44]:
img = cv.imread("C:/Users/Alaa/Data_Science/Projects/graduation/action-net/Action-Net-master/images/video.jpg")
prepared = prepare(img)

In [29]:
cv.imshow("prepared", prepared)
if cv.waitKey(0) and 0xFF == ord('q'):
    cv.destroyAllWindows()

In [30]:
cv.destroyAllWindows()

In [33]:
prepared.shape

(360, 480, 3)

In [45]:
# img = "C:/Users/Alaa/Data_Science/Projects/graduation/action-net/Action-Net-master/images/3.jpg"

# img_data = Image.open(img)
# img_arr = np.array(img_data)
img_arr = np.resize(prepared, (224, 224, 3))
img_arr = np.expand_dims(img_arr, axis=0)
# print(img_arr)
# print(img_data)
predictions = model.predict(img_arr)[0]
print(predictions)
print(model_classes[np.argmax(predictions)])

[0.7040606  0.29593942]
fight


In [9]:
print(img_arr.shape)

(1, 224, 224, 3)


In [8]:
# img = "C:/Users/Alaa/Data_Science/Projects/graduation/action-net/Action-Net-master/images/3.jpg"
# img = image.load_img(img) #, target_size=(img_width, img_height)
# x = image.img_to_array(img)
# x = np.expand_dims(x, axis=0)

# live video

In [46]:
import cv2 as cv
import numpy as np
from PIL import ImageGrab

In [11]:
import time

In [49]:
while True:
    screen =  np.array(ImageGrab.grab(bbox=(25,160,640,500)))
    #print('Frame took {} seconds'.format(time.time()-last_time))
    screen = cv.cvtColor(screen, cv.COLOR_BGR2RGB)
#     cv.imshow('window', screen)
    
#     img_data = Image.open(img)
#     img_arr = np.array(img_data)
    img_arr = prepare(screen)
    img_arr = np.resize(screen, (224, 224, 3))
    img_arr = np.expand_dims(img_arr, axis=0)
    # print(img_arr)
    # print(img_data)
    predictions = model.predict(img_arr)[0]
    print("highest prediction is")
    print(model_classes[np.argmax(predictions)])
    print("probability : ", predictions[np.argmax(predictions)])
    #cv2.imshow('window',cv2.cvtColor(screen, cv2.COLOR_BGR2RGB))
#     time.sleep(0.5)
#     if cv.waitKey(25) & 0xFF == ord('q'):
#         cv.destroyAllWindows()
#         break


highest prediction is
fight
probability :  0.73998994
highest prediction is
fight
probability :  0.73998994
highest prediction is
fight
probability :  0.73998994
highest prediction is
fight
probability :  0.73998994
highest prediction is
fight
probability :  0.73998994
highest prediction is
fight
probability :  0.73998994
highest prediction is
fight
probability :  0.72244525
highest prediction is
fight
probability :  0.7205322
highest prediction is
fight
probability :  0.699754
highest prediction is
fight
probability :  0.66543114
highest prediction is
fight
probability :  0.728043
highest prediction is
fight
probability :  0.7033274
highest prediction is
fight
probability :  0.6804619
highest prediction is
fight
probability :  0.6482421
highest prediction is
fight
probability :  0.6525237
highest prediction is
fight
probability :  0.7365415
highest prediction is
fight
probability :  0.661928
highest prediction is
nofight
probability :  0.5016981
highest prediction is
fight
probability

highest prediction is
fight
probability :  0.5389466
highest prediction is
fight
probability :  0.5186239
highest prediction is
fight
probability :  0.5189902
highest prediction is
fight
probability :  0.525202
highest prediction is
fight
probability :  0.50150025
highest prediction is
fight
probability :  0.53209186
highest prediction is
fight
probability :  0.57207906
highest prediction is
fight
probability :  0.54440457
highest prediction is
fight
probability :  0.5374524
highest prediction is
nofight
probability :  0.5112592
highest prediction is
fight
probability :  0.501377
highest prediction is
fight
probability :  0.59558177
highest prediction is
fight
probability :  0.60966676
highest prediction is
fight
probability :  0.61028343
highest prediction is
fight
probability :  0.5453149
highest prediction is
fight
probability :  0.6479447
highest prediction is
fight
probability :  0.6207412
highest prediction is
fight
probability :  0.64040416
highest prediction is
fight
probabilit

highest prediction is
fight
probability :  0.50244933
highest prediction is
nofight
probability :  0.5420218
highest prediction is
nofight
probability :  0.5169467
highest prediction is
nofight
probability :  0.5256963
highest prediction is
nofight
probability :  0.55016565
highest prediction is
nofight
probability :  0.5956609
highest prediction is
nofight
probability :  0.6034952
highest prediction is
nofight
probability :  0.58476776
highest prediction is
fight
probability :  0.50338143
highest prediction is
nofight
probability :  0.6163413
highest prediction is
nofight
probability :  0.5205165
highest prediction is
nofight
probability :  0.60700494
highest prediction is
nofight
probability :  0.67648643
highest prediction is
nofight
probability :  0.649375
highest prediction is
nofight
probability :  0.6018525
highest prediction is
nofight
probability :  0.5876758
highest prediction is
nofight
probability :  0.5279505
highest prediction is
nofight
probability :  0.5032228
highest p

highest prediction is
fight
probability :  0.55049556
highest prediction is
fight
probability :  0.68829805
highest prediction is
fight
probability :  0.67264247
highest prediction is
fight
probability :  0.67322683
highest prediction is
nofight
probability :  0.5704284
highest prediction is
fight
probability :  0.7267525
highest prediction is
fight
probability :  0.68697274
highest prediction is
fight
probability :  0.54241186
highest prediction is
nofight
probability :  0.5752489
highest prediction is
nofight
probability :  0.683704
highest prediction is
fight
probability :  0.5624409
highest prediction is
nofight
probability :  0.5148382
highest prediction is
nofight
probability :  0.6623115
highest prediction is
nofight
probability :  0.5019158
highest prediction is
nofight
probability :  0.6477588
highest prediction is
fight
probability :  0.6093868
highest prediction is
fight
probability :  0.6343465
highest prediction is
fight
probability :  0.6574683
highest prediction is
fight

highest prediction is
nofight
probability :  0.5310311
highest prediction is
nofight
probability :  0.55908453
highest prediction is
fight
probability :  0.6653208
highest prediction is
fight
probability :  0.6153398
highest prediction is
fight
probability :  0.7044634
highest prediction is
fight
probability :  0.6636659
highest prediction is
fight
probability :  0.59886634
highest prediction is
nofight
probability :  0.58629286
highest prediction is
fight
probability :  0.7358232
highest prediction is
fight
probability :  0.6997428
highest prediction is
fight
probability :  0.6963818
highest prediction is
fight
probability :  0.70595944
highest prediction is
fight
probability :  0.68880594
highest prediction is
fight
probability :  0.63850164
highest prediction is
fight
probability :  0.67740446
highest prediction is
fight
probability :  0.6627618
highest prediction is
fight
probability :  0.61005193
highest prediction is
nofight
probability :  0.56405336
highest prediction is
fight
p

highest prediction is
nofight
probability :  0.6899322


KeyboardInterrupt: 

## Single Image

In [13]:
screen =  np.array(ImageGrab.grab(bbox=(25,160,640,500)))
#print('Frame took {} seconds'.format(time.time()-last_time))
screen = cv.cvtColor(screen, cv.COLOR_BGR2RGB)
cv.imshow('window', screen)
img_arr = np.resize(screen, (224, 224, 3))
img_arr = np.expand_dims(img_arr, axis=0)
predictions = model.predict(img_arr)[0]
print("highest prediction is")
print(model2_classes[np.argmax(predictions)])
print("probability : ", predictions[np.argmax(predictions)])
cv.waitKey(0)
cv.destroyAllWindows()

highest prediction is
fighting
probability :  0.29728484
